
# 0. Pseudo Data Generator 

- Output: (id, timestamp, data vec of 1s (100x3))

# 1. Data Stream (? + Data Preprocessing)

- Input: (id, timestamp, data vec of 1s (100x3))

- Process: Data buffer (Kafka) 

- Output: [(id, timestamp, data vec of 30s (3000x3)), (id, timestamp, data vec of 30s (3000x3)), ... ] for every second

# 2. Phase Picking

- Preload: NN model of PhaseNet

- Input: [(id, timestamp, data vec (3000x3)), (id, timestamp, data vec of (3000x3)), ... ]

- Process: PhaseNet 

<!-- - Output: (id, timestamp, idx_p, prob_p, amp_p, idx_s, prob_s, amp_s) -->
- Output: [(id, timestamp, idx(int), prob(float), amp(float), type(string)), ...]

# 3. Data Stream

<!-- - Input: (id, timestamp, idx_p, prob_p, amp_p, idx_s, prob_s, amp_s) of overlapping 30s -->
- Input: [(id, timestamp, idx, prob, amp, type), ...] of overlapping 30s

- Process: Data buffer (Kafka) 

<!-- - Output: (id, timestamp, idx_p, prob_p, amp_p, idx_s, prob_s, amp_s) with no overlapping and reprot the recent 10 s. -->
- Output: [(id, timestamp, idx, prob, amp, type), ...] with no overlapping and reprot the recent 10 s. 

# 4. Association

- Preload: csv file of seismic station information (id, latitude, longitude, elevation)

<!-- - Input: (id, timestamp, idx_p, prob_p, amp_p, idx_s, prob_s, amp_s) -->
- Input: [(id, timestamp, idx, prob, amp, type), ...]

- Process: GMMA

- Output: [(timestamp, latitude, longtidue, depth, magnitude), ...] of earthquakes

# 5. Data Stream

- Input: [(timestamp, latitude, longtidue, depth, magnitude), ...]

- Process: Data buffer (Kafka) 

- Output: [(timestamp, latitude, longtidue, depth, magnitude), ...]

# 6. Visulization and Notification

- Input: [(timestamp, latitude, longtidue, depth, magnitude), ...] of earthquakes

- WebUI + Twitter Bot

## Code:

## 1. Data

In [93]:
import obspy
import pandas as pd
import os
import datetime
import numpy as np
import time

In [52]:
waveform_path = "./data/waveforms"
fnames = pd.read_csv("fname.csv")
stations = pd.read_csv("stations.csv", sep="\t")

In [98]:
window_size = 100
sampling_rate = 100
dtype = "float32"
timestamp = lambda x: x.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]

for i, row in fnames.iterrows():
    print(row.loc["fname"])
    meta = obspy.read(os.path.join(waveform_path, row.loc["fname"]))
    assert(meta[0].stats.sampling_rate == sampling_rate)
    meta = meta.merge(fill_value=0)
    start_time = min([st.stats.starttime for st in meta])
    end_time = max([st.stats.endtime for st in meta])
    meta = meta.trim(start_time, end_time, pad=True, fill_value=0)
    num_station = len(stations)
    nt = len(meta[0].data)
    data = np.zeros([num_station, nt, 3], dtype=dtype)
    station_id = []
    for i in range(num_station):
        sta = stations.iloc[i]["station"]
        station_id.append(sta)
        comp = stations.iloc[i]["component"].split(",")
        resp = stations.iloc[i]["response"].split(",")
        for j in range(len(comp)):
            data[i, :, j] = meta.select(id=sta+comp[j])[0].data.astype(dtype) / float(resp[j])
    idx = 0
    while idx < nt:
        for i in range(num_station):
            print (station_id[i], timestamp(start_time+idx/sampling_rate), data[i, idx:idx+window_size, :].shape)
        idx += window_size
        time.sleep(1)
        if idx > 3*window_size:
            raise

2020-10-01T00.mseed
CI.BOM..HH 2020-10-01T00:00:00.003 (100, 3)
CI.COA..HH 2020-10-01T00:00:00.003 (100, 3)
CI.CRR..HH 2020-10-01T00:00:00.003 (100, 3)
CI.DRE..HH 2020-10-01T00:00:00.003 (100, 3)
CI.ERR..HH 2020-10-01T00:00:00.003 (100, 3)
CI.FRK..HH 2020-10-01T00:00:00.003 (100, 3)
CI.IMP..HH 2020-10-01T00:00:00.003 (100, 3)
CI.RXH..HH 2020-10-01T00:00:00.003 (100, 3)
CI.SAL..HH 2020-10-01T00:00:00.003 (100, 3)
CI.SGL..HH 2020-10-01T00:00:00.003 (100, 3)
CI.SLV..HH 2020-10-01T00:00:00.003 (100, 3)
CI.SNR..HH 2020-10-01T00:00:00.003 (100, 3)
CI.SWS..HH 2020-10-01T00:00:00.003 (100, 3)
CI.WES..HH 2020-10-01T00:00:00.003 (100, 3)
CI.WMD..HH 2020-10-01T00:00:00.003 (100, 3)
CI.YUH2..HH 2020-10-01T00:00:00.003 (100, 3)
CI.BOM..HH 2020-10-01T00:00:01.003 (100, 3)
CI.COA..HH 2020-10-01T00:00:01.003 (100, 3)
CI.CRR..HH 2020-10-01T00:00:01.003 (100, 3)
CI.DRE..HH 2020-10-01T00:00:01.003 (100, 3)
CI.ERR..HH 2020-10-01T00:00:01.003 (100, 3)
CI.FRK..HH 2020-10-01T00:00:01.003 (100, 3)
CI.IMP..HH 

RuntimeError: No active exception to reraise